In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
joint_2ds = {'train': [], 'test': []}
joint_2d_from_canonical_3ds = {'train': [], 'test': []}
confidences = {'train': [], 'test': []}
#joint3d_images = {'train': [], 'test': []}
#joints_25d_images = {'train': [], 'test': []}
#_25d_factors = {'train': [], 'test': []}
camera_names = {'train': [], 'test': []}
actions = {'train': [], 'test': []}
sources = {'train': [], 'test': []}
frames = {'train': [], 'test': []}
#world_3ds = {'train': [], 'test': []}
cam_3ds = {'train': [], 'test': []}
cam_3d_from_canonical_3ds = {'train': [], 'test': []}
cam_params = {'train': [], 'test': []}

# target 
pkl_folder = '/home/hrai/codes/MotionBERT/data/motion3d/3dhp_gt_test_canonical_3d_with_fixed_dist_5_pkl'
test_subject = ['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']
only_test = True

blacklist = []

for item in os.listdir(pkl_folder):
    print(item)
    subject = item.split('.')[0]
    
    # filtering
    #if action not in action_list: continue 
    #if int(cam_num) not in camera_list: continue
    if only_test:
        if subject not in test_subject: continue
    if subject in blacklist: continue
    
    item_path = os.path.join(pkl_folder, item) 
    #print(item_path)
    data = readpkl(item_path) 
    if subject in test_subject: data_type = 'test' 
    else: data_type = 'train' 
    joint_2ds[data_type] += list(np.array(data['joint_2d'])) 
    joint_2d_from_canonical_3ds[data_type] += list(np.array(data['joint_2d_from_canonical_3d']))
    confidences[data_type] += list(np.array(data['confidence'])) 
    camera_names[data_type] += list(data['camera_name'])
    actions[data_type] += list(data['action'])
    sources[data_type] += list(data['source'])
    frames[data_type] += list(data['frame'])
    cam_3ds[data_type] += list(np.array(data['cam_3d'])) # mm to m
    cam_3d_from_canonical_3ds[data_type] += list(np.array(data['cam_3d_from_canonical_3d'])) # mm to m
    #joint3d_images[data_type] += list(np.array(data['joint3d_image']))
    #world_3ds[data_type] += list(np.array(data['world_3d']))
    cam_params[data_type] += list(data['cam_param'])
    
    #print(cam_3ds[data_type ][0])
    
data_total = {'train': {}, 'test': {}}

for data_type in ['train', 'test']:
    data_total[data_type]['joint_2d']          = copy.deepcopy(np.array(joint_2ds[data_type]))
    data_total[data_type]['joint_2d_from_canonical_3d'] = copy.deepcopy(np.array(joint_2d_from_canonical_3ds[data_type]))
    data_total[data_type]['confidence']        = copy.deepcopy(np.array(confidences[data_type]))
    data_total[data_type]['camera_name']       = copy.deepcopy(np.array(camera_names[data_type]))
    data_total[data_type]['action']            = copy.deepcopy(np.array(actions[data_type]))
    data_total[data_type]['source']            = copy.deepcopy(np.array(sources[data_type]))
    data_total[data_type]['frame']             = copy.deepcopy(np.array(frames[data_type]))
    data_total[data_type]['cam_3d']            = copy.deepcopy(np.array(cam_3ds[data_type]))
    #data_total[data_type]['world_3d']          = copy.deepcopy(np.array(world_3ds[data_type]))
    #data_total[data_type]['joint3d_image']     = copy.deepcopy(np.array(joint3d_images[data_type]))
    data_total[data_type]['cam_3d_from_canonical_3d'] = copy.deepcopy(np.array(cam_3d_from_canonical_3ds[data_type]))
    data_total[data_type]['cam_param']         = copy.deepcopy(np.array(cam_params[data_type]))


if (len(data_total['train']['joint_2d']) == 0) or only_test:
    for key in data_total['train'].keys():
        data_total['train'][key] = copy.deepcopy(data_total['test'][key][:243])
        print(key, data_total['train'][key].shape)
        
data_total['train']['joint_2d'].shape, data_total['test']['joint_2d'].shape  

TS4.pkl
TS5.pkl
TS6.pkl
TS2.pkl
TS3.pkl
TS1.pkl
joint_2d (243, 17, 2)
joint_2d_from_canonical_3d (243, 17, 3)
confidence (243, 17, 1)
camera_name (243,)
action (243,)
source (243,)
frame (243,)
cam_3d (243, 17, 3)
cam_3d_from_canonical_3d (243, 17, 3)
cam_param (243,)


((243, 17, 2), (24888, 17, 2))

In [11]:
data_total['test']['joint_2d'].shape, data_total['test']['confidence'].shape  

((24888, 17, 2), (24888, 17, 1))

In [12]:
save_path = os.path.join('/home/hrai/codes/MotionBERT', f'data/motion3d/3dhp_gt_test_canonical_3d_from_fixed_dist_5_TS1_6.pkl')
savepkl(data_total, save_path)